In [ ]:
from autogen import AssistantAgent, UserProxyAgent

# Supervisor (LLM-based)
supervisor = AssistantAgent(
    name="supervisor",
    system_message="Analyze seismic data by coordinating fault detection and horizon picking. Validate results.",
    llm_config={"config_list": [...]},  # LLM 模型配置 (e.g., OpenAI, LiteLLM)
)

# User proxy for human input
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    code_execution_config={"work_dir": "seismic"},
)

# Register tools for models
user_proxy.register_function(
    function_map={
        "fault_detection": fault_detection, # 断层检测代理
        "horizon_picking": horizon_picking, # 地层拾取代理
    }
)

In [ ]:
user_proxy.initiate_chat(
    supervisor,
    message="Analyze the seismic data at path 'data/example.segy'. First detect faults, then pick horizons.",
)

In [ ]:
def fault_detection(seismic_data: str) -> dict:
    # Load your model and run inference
    return {"fault_locations": [...]}

def horizon_picking(seismic_data: str) -> dict:
    # Run horizon picking model
    return {"horizons": [...]}

In [ ]:
import autogen
from typing import List, Dict, Any
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Configuration for local LLM endpoints
def get_llm_config(model_path: str, model_type: str = "llama") -> Dict[str, Any]:
    config = {
        "cache_seed": 42,
        "temperature": 0.7,
        "max_tokens": 2000,
        "model": model_type,
        "local_path": model_path,
    }
    return config

# Custom LLM wrapper for local models
class LocalLLMWrapper:
    def __init__(self, model_path: str, model_type: str = "llama"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.model_type = model_type

    def generate(self, prompt: str, max_tokens: int = 2000) -> str:
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        outputs = self.model.generate(
            inputs["input_ids"],
            max_new_tokens=max_tokens,
            temperature=0.7,
            do_sample=True
        )
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response

# Define agent configurations
def create_agents(local_llm_configs: List[Dict[str, Any]]):
    # Supervisor agent using local LLama/Qwen/Phi
    supervisor_config = {
        "llm_config": local_llm_configs[0],  # Using first model as supervisor
        "system_message": """You are a supervisor for seismic interpretation tasks.
        Your role is to coordinate between different specialist agents and ensure
        the interpretation process follows best practices."""
    }
    
    # Seismic data preprocessor agent
    preprocessor_config = {
        "llm_config": local_llm_configs[1],  # Using second model
        "system_message": """You are a specialist in seismic data preprocessing.
        Your tasks include data loading, noise reduction, and signal enhancement."""
    }
    
    # Seismic feature extraction agent
    feature_extractor_config = {
        "llm_config": local_llm_configs[2],  # Using third model
        "system_message": """You are a specialist in seismic feature extraction.
        Your role is to identify and extract key seismic attributes and patterns."""
    }

    # Create agents
    supervisor = autogen.AssistantAgent(
        name="SupervisorAgent",
        **supervisor_config
    )
    
    preprocessor = autogen.AssistantAgent(
        name="PreprocessorAgent",
        **preprocessor_config
    )
    
    feature_extractor = autogen.AssistantAgent(
        name="FeatureExtractorAgent",
        **feature_extractor_config
    )
    
    # User proxy agent for interaction
    user_proxy = autogen.UserProxyAgent(
        name="UserProxy",
        human_input_mode="TERMINATE",
        max_consecutive_auto_reply=10
    )
    
    return supervisor, preprocessor, feature_extractor, user_proxy

# Main workflow
def main():
    # Setup local model configurations
    model_configs = [
        get_llm_config("/path/to/llama-model", "llama"),
        get_llm_config("/path/to/qwen-model", "qwen"),
        get_llm_config("/path/to/phi-model", "phi")
    ]
    
    # Initialize local LLM instances
    llm_instances = [
        LocalLLMWrapper(config["local_path"], config["model"])
        for config in model_configs
    ]
    
    # Create agents
    supervisor, preprocessor, feature_extractor, user_proxy = create_agents(model_configs)
    
    # Define group chat
    groupchat = autogen.GroupChat(
        agents=[supervisor, preprocessor, feature_extractor, user_proxy],
        messages=[],
        max_round=50
    )
    
    # Create group chat manager
    manager = autogen.GroupChatManager(
        groupchat=groupchat,
        llm_config=model_configs[0]  # Using supervisor's config
    )
    
    # Example: Initiate a seismic interpretation task
    user_proxy.initiate_chat(
        manager,
        message="""Please analyze the seismic data in file 'example.sgy':
        1. Preprocess the data to remove noise
        2. Extract key features and horizons
        3. Provide interpretation results"""
    )

if __name__ == "__main__":
    main()